# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Google API URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv = '../WeatherPy/Output/weather_df.csv'
weather_df = pd.read_csv(weather_csv)

weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (°C),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,butaritari,3.0707,172.7902,27.89,75,7,6.83,KI,1643979875
1,1,karratha,-20.7377,116.8463,29.69,69,86,4.62,AU,1643979658
2,2,bambous virieux,-20.2567,57.4061,26.74,77,100,7.65,MU,1643979875
3,3,mahebourg,-20.4081,57.7000,27.36,81,100,7.15,MU,1643979876
4,4,hasaki,35.7333,140.8333,0.22,93,100,4.99,JP,1643979876


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Lat & Long as locations, humidity as weight
locations = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity'].astype(float)
maxhumidity = humidity.max()

In [6]:
# Generate map and plot heatlayer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 max_intensity=maxhumidity,
                                 point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_weather_df = weather_df[(weather_df['Max Temperature (°C)'] < 25) & 
                             (weather_df['Humidity'] < 35) & 
                             (weather_df['Cloudiness'] > 20)]

ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (°C),Humidity,Cloudiness,Wind Speed,Country,Date
87,87,arlit,18.7369,7.3853,24.89,8,50,1.72,NE,1643979902
297,297,cockburn town,-32.0833,141.0000,22.69,27,100,6.13,AU,1643979973
329,329,gat,31.6100,34.7642,13.38,12,97,8.07,IL,1643979983
367,367,jinchang,38.4953,102.1739,-6.66,33,100,1.76,CN,1643979960
382,382,hami,42.8000,93.4500,-3.17,33,94,2.28,CN,1643979961
426,426,sakakah,29.9697,40.2064,14.88,30,31,8.72,SA,1643980015
458,458,sonoita,31.8500,-112.8333,5.29,16,38,7.29,MX,1643980026


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_weather_df

In [9]:
# Parameter dictionary
params = {
    "types": "lodging",
    "radius": 5000,
    "keyword": "hotel",
    "key": gkey,
}

#prevent SettingWithCopyWarning message from appearing
pd.options.mode.chained_assignment = None

# Loop through DF to identify hotels
for index, row in hotel_df.iterrows():
    city_name = row['City']
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = f'{lat},{lng}'
    print(f'Fetching hotel for {city_name}.')
    response = requests.get(base_url, params=params).json()
    hotel_name = response['results'][0]['name']
    try:
        print(f'Closest hotel in {city_name}: {hotel_name}.')
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
    except (KeyError, IndexError):
        print('No Hotel Result. Skipping to next...')

Fetching hotel for arlit.
Closest hotel in arlit: Hôtel Temet.
Fetching hotel for cockburn town.
Closest hotel in cockburn town: Coburn Hotel.
Fetching hotel for gat.
Closest hotel in gat: Desert Gat.
Fetching hotel for jinchang.
Closest hotel in jinchang: Jinchuan Hotel.
Fetching hotel for hami.
Closest hotel in hami: Π Hotel.
Fetching hotel for sakakah.
Closest hotel in sakakah: Raoum Inn Hotel.
Fetching hotel for sonoita.
Closest hotel in sonoita: Hotel Excelsior.


In [10]:
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (°C),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
87,87,arlit,18.7369,7.3853,24.89,8,50,1.72,NE,1643979902,Hôtel Temet
297,297,cockburn town,-32.0833,141.0000,22.69,27,100,6.13,AU,1643979973,Coburn Hotel
329,329,gat,31.6100,34.7642,13.38,12,97,8.07,IL,1643979983,Desert Gat
367,367,jinchang,38.4953,102.1739,-6.66,33,100,1.76,CN,1643979960,Jinchuan Hotel
382,382,hami,42.8000,93.4500,-3.17,33,94,2.28,CN,1643979961,Π Hotel
426,426,sakakah,29.9697,40.2064,14.88,30,31,8.72,SA,1643980015,Raoum Inn Hotel
458,458,sonoita,31.8500,-112.8333,5.29,16,38,7.29,MX,1643980026,Hotel Excelsior


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))